In [130]:
import pandas as pd
from glob import glob
import re
import os


In [131]:
# 全行全列表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# 小数点以下n桁表示
pd.options.display.float_format = '{:.2f}'.format

In [132]:
csv_files = glob('../csv/EXA_FIRST_マイジャグラーV_*.csv')
len(csv_files), csv_files[0]

(162, '../csv\\EXA_FIRST_マイジャグラーV_2024-04-01.csv')

In [133]:
def extract_information(csv_file):
    pattern = r"(.+)_(.+)_(\d{4}-\d{2}-\d{2}).csv"

    file_name = os.path.basename(csv_file)
    # print(file_name)
    match = re.match(pattern, file_name)

    if match:
        hall_name, machine_name, date_str = match.groups()
        return hall_name, machine_name, date_str
    else:
        print(f"ファイル名の形式が一致しません {file_name}")
        print(f"ホール名: {hall_name}")
        print(f"機種名: {machine_name}")
        print(f"日付: {date_str}")


In [134]:
def calc_grape_rate(game_count, bb_count, rb_count, coins):
    bb_coins = 240
    rb_coins = 96
    cherry_coins = 2
    cherry_rate = 38.1
    replay_coins = 3
    replay_rate = 7.3
    grape_coins = 8

    in_coins = game_count * 3

    out_with_cherry = (
        bb_coins * bb_count
        + rb_coins * rb_count
        + (cherry_coins * game_count / cherry_rate)
        + (replay_coins * game_count / replay_rate)
    )
    without_cherry_rate = 0.667
    out_without_cherry = (
        bb_coins * bb_count
        + rb_coins * rb_count
        + (cherry_coins * game_count / cherry_rate * without_cherry_rate)
        + (replay_coins * game_count / replay_rate)
    )

    grape_rate_with_cherry = (
        grape_coins * game_count / (coins + in_coins - out_with_cherry)
    )
    grape_rate_without_cherry = (
        grape_coins * game_count / (coins + in_coins - out_without_cherry)
    )

    # print("フリー打ち : ", grape_rate_without_cherry)
    # print("チェリー狙い : ", grape_rate_with_cherry)
    
    return grape_rate_with_cherry, grape_rate_without_cherry


def calc_total_coins(game_count, bb_count, rb_count, grape_rate=5.9):

    BB_COINS = 240
    RB_COINS = 96
    CHERRY_COINS = 2
    CHERRY_RATE = 38.1
    REPLAY_COINS = 3
    REPLAY_RATE = 7.3
    GRAPE_COINS = 8

    in_coins = game_count * 3

    out_coins = sum([
        BB_COINS * bb_count,
        RB_COINS * rb_count,
        CHERRY_COINS * game_count / CHERRY_RATE,
        REPLAY_COINS * game_count / REPLAY_RATE,
        GRAPE_COINS * game_count / grape_rate,
        ])
    total_coins = out_coins - in_coins
    # print(total_coins)

    return total_coins

In [135]:
dataframes = []
for csv_file in csv_files:
    hall_name, machine_name, date_str = extract_information(csv_file)
    df = pd.read_csv(csv_file, encoding="utf-8-sig")
    df["date"] = date_str
    df["hall_name"] = hall_name
    df["machine_name"] = machine_name
    dataframes.append(df)
    
if dataframes:
    df = pd.concat(dataframes, ignore_index=True)
    df.to_csv('../analysis_data.csv', index=False)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13107 entries, 0 to 13106
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   台番号           13107 non-null  int64  
 1   G数            13107 non-null  object 
 2   差枚            13107 non-null  object 
 3   BB            13107 non-null  int64  
 4   RB            13026 non-null  float64
 5   合成確率          13107 non-null  object 
 6   BB確率          13107 non-null  object 
 7   RB確率          13026 non-null  object 
 8   date          13107 non-null  object 
 9   hall_name     13107 non-null  object 
 10  machine_name  13107 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 1.1+ MB


In [136]:
df.head()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name
0,1033,"8,048","+1,597",40,17.00,1/141.2,1/201.2,1/473.4,2024-04-01,EXA_FIRST,マイジャグラーV
1,1034,"4,791",-179,19,9.00,1/171.1,1/252.2,1/532.3,2024-04-01,EXA_FIRST,マイジャグラーV
2,1035,"6,810","+1,253",31,22.00,1/128.5,1/219.7,1/309.5,2024-04-01,EXA_FIRST,マイジャグラーV
3,1036,"7,367",-44,28,22.00,1/147.3,1/263.1,1/334.9,2024-04-01,EXA_FIRST,マイジャグラーV
4,1037,"8,379","+2,126",38,26.00,1/130.9,1/220.5,1/322.3,2024-04-01,EXA_FIRST,マイジャグラーV


In [137]:
df["G数"] = (
    df["G数"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["差枚"] = (
    df["差枚"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["RB"] = (
    df["RB"]
    .astype(str)  # 文字列型に変換
    .str.replace(",", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(int)  # 整数型に変換
)
df["合成確率"] = (
    df["合成確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["BB確率"] = (
    df["BB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)
df["RB確率"] = (
    df["RB確率"]
    .astype(str)  # 文字列型に変換
    .str.replace("1/", "", regex=True)  # カンマを削除
    .str.extract(r"(\d+)")  # 数字部分のみ抽出
    .dropna()  # NaNを削除
    .astype(float)  # 整数型に変換
)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.month.astype(int)
df['weekday'] = df['date'].dt.weekday.astype(int)
df['day'] = df['date'].dt.day.astype(int)
df["last_digit_of_day"] = df["date"].dt.day.astype(str).str[-1].astype(int)
df["g_cherry"], df["g_free"] = calc_grape_rate(df["G数"], df["BB"], df["RB"], df["差枚"])
df["total_coins"] = calc_total_coins(df["G数"], df["BB"], df["RB"])

df.to_csv('../analysis_data.csv', index=False)


In [138]:
df.head()

,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率,date,hall_name,machine_name,month,weekday,day,last_digit_of_day,g_cherry,g_free,total_coins
0,1033,8048,1597,40,17.00,141.00,201.00,473.00,2024-04-01,EXA_FIRST,マイジャグラーV,4,0,1,1,5.97,5.90,1730.41
1,1034,4791,179,19,9.00,171.00,252.00,532.00,2024-04-01,EXA_FIRST,マイジャグラーV,4,0,1,1,5.55,5.48,-232.33
2,1035,6810,1253,31,22.00,128.00,219.00,309.00,2024-04-01,EXA_FIRST,マイジャグラーV,4,0,1,1,6.07,5.99,1512.01
3,1036,7367,44,28,22.00,147.00,263.00,334.00,2024-04-01,EXA_FIRST,マイジャグラーV,4,0,1,1,5.95,5.88,134.41
4,1037,8379,2126,38,26.00,130.00,220.00,322.00,2024-04-01,EXA_FIRST,マイジャグラーV,4,0,1,1,5.70,5.63,1723.62


In [181]:
target_df = df[df["day"] == 1]
game_count = target_df.pivot_table(index="台番号", values="G数", aggfunc="sum")
# coin_count = target_df.pivot_table(index="台番号", values="差枚", aggfunc="sum")
bb_count = target_df.pivot_table(index="台番号", values="BB", aggfunc="sum")
rb_count = target_df.pivot_table(index="台番号", values="RB", aggfunc="sum")
df_day = pd.concat([game_count, bb_count, rb_count], axis=1)
df_day['BB_rate'] = df_day['G数'] / df_day["BB"]
df_day['RB_rate'] = df_day['G数'] / df_day["RB"]
df_day['TOTAL_rate'] = df_day['G数'] / (df_day["RB"]+df_day["BB"])
# df_day["total_coins"] = calc_total_coins(df_day["G数"], df_day["BB"], df_day["RB"])

df_filter = df_day[
    (df_day["RB_rate"] <= 300) & (df_day["TOTAL_rate"] <= 150) & (df_day["G数"] >= 10000)
    ].sort_values(["RB_rate",], ascending=[True])
df_filter

,G数,BB,RB,BB_rate,RB_rate,TOTAL_rate
台番号,,,,,,
1086,80414,303,317.00,265.39,253.67,129.70
1078,76638,305,289.00,251.27,265.18,129.02
1070,71642,256,260.00,279.85,275.55,138.84
1039,78374,307,284.00,255.29,275.96,132.61
1077,85826,341,310.00,251.69,276.86,131.84
1016,64837,239,233.00,271.28,278.27,137.37
1090,77700,290,279.00,267.93,278.49,136.56
1010,72586,292,260.00,248.58,279.18,131.50
1076,83589,369,299.00,226.53,279.56,125.13


In [182]:
table = target_df[target_df['台番号'].isin(df_filter.index)].pivot_table(index="台番号", columns="date", values=["RB確率"], margins=True)
table

RB確率                                          \
date 2024-04-01 00:00:00 2024-05-01 00:00:00 2024-06-01 00:00:00   
台番号                                                                
1010                 NaN              327.00              229.00   
1016                 NaN              204.00              376.00   
1017                 NaN              729.00              227.00   
1018                 NaN              563.00              256.00   
1023                 NaN              312.00              280.00   
1026                 NaN              236.00              421.00   
1029                 NaN              194.00              186.00   
1036              334.00              270.00              247.00   
1039              219.00              337.00              533.00   
1040              256.00              468.00              206.00   
1044              311.00              309.00              345.00   
1046              426.00              467.00              351.00   
1047              238.00              348.00              283.00   
1070              175.00              224.00              531.00   
1072              307.00              286.00              210.00   
1074              325.00              429.00              302.00   
1076              326.00              254.00              326.00   
1077              266.00              602.00              224.00   
1078              191.00              405.00              313.00   
1079              251.00              318.00              249.00   
1085              181.00              471.00              581.00   
1086              315.00              275.00              140.00   
1090              306.00              439.00              258.00   
1097              220.00              243.00              393.00   
1098              314.00              278.00              288.00   
All               275.61              359.52              310.20   

                                                                  \
date 2024-07-01 00:00:00 2024-08-01 00:00:00 2024-09-01 00:00:00   
台番号                                                                
1010              318.00              293.00              264.00   
1016              393.00              340.00              271.00   
1017              250.00              377.00              512.00   
1018              689.00              246.00              205.00   
1023              296.00              338.00              205.00   
1026              421.00              308.00              332.00   
1029              223.00              334.00              295.00   
1036              658.00              261.00              411.00   
1039              291.00              257.00              254.00   
1040              408.00              225.00              223.00   
1044              247.00              224.00              224.00   
1046              253.00              743.00              141.00   
1047              484.00              452.00              193.00   
1070              360.00              250.00              352.00   
1072              499.00              341.00              313.00   
1074              214.00              337.00              335.00   
1076              214.00              343.00              303.00   
1077              286.00              331.00              365.00   
1078              234.00              436.00              244.00   
1079              258.00              615.00              348.00   
1085              214.00              293.00              262.00   
1086              198.00              228.00              353.00   
1090              422.00              364.00              288.00   
1097              222.00              295.00              508.00   
1098              432.00              192.00              321.00   
All               339.36              336.92              300.88   

                                                                  \
date 2